In [1]:
import LeModel
import pandas as pd
import cv2
from matplotlib import pyplot as plt

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
df = pd.read_csv('flora.csv')
df['filename'] = '/home/haroldle/Downloads/dataset/' + df['ID'].astype(str).str.zfill(4) + '.jpg'
df

,Unnamed: 0,SPECIES,ID,SET,filename
0,297,Manilkara elata,3135,train,/home/haroldle/Downloads/dataset/3135.jpg
1,137,Mezilaurus itauba,3307,train,/home/haroldle/Downloads/dataset/3307.jpg
2,1138,Araucaria angustifolia,223,train,/home/haroldle/Downloads/dataset/0223.jpg
3,1333,Swietenia macrophylla,4509,train,/home/haroldle/Downloads/dataset/4509.jpg
4,186,Machaerium paraguariense,2929,train,/home/haroldle/Downloads/dataset/2929.jpg
...,...,...,...,...,...
1896,905,Peltogyne sp,4116,validation,/home/haroldle/Downloads/dataset/4116.jpg
1897,1791,Hymenaea sp,2413,validation,/home/haroldle/Downloads/dataset/2413.jpg
1898,1096,Araucaria angustifolia,205,validation,/home/haroldle/Downloads/dataset/0205.jpg
1899,235,Dipteryx sp,1606,validation,/home/haroldle/Downloads/dataset/1606.jpg


In [3]:
# test Le's model
le = LeModel.LeModel()

valid = df.loc[df['SET']=='validation']
for idx in range(10):
    print(df['filename'].values[idx])
    im = cv2.imread(df['filename'].values[idx])
    #plt.imshow(im)
    #plt.show()
    print(le.predict(im))

/home/haroldle/Downloads/dataset/3135.jpg
['Manilkara elata']
/home/haroldle/Downloads/dataset/3307.jpg
['Mezilaurus itauba']
/home/haroldle/Downloads/dataset/0223.jpg
['Araucaria angustifolia']
/home/haroldle/Downloads/dataset/4509.jpg
['Swietenia macrophylla']
/home/haroldle/Downloads/dataset/2929.jpg
['Machaerium paraguariense']
/home/haroldle/Downloads/dataset/3515.jpg
['Mimosa scabrella']
/home/haroldle/Downloads/dataset/2718.jpg
['Inga vera']
/home/haroldle/Downloads/dataset/1821.jpg
['Mimosa scabrella']
/home/haroldle/Downloads/dataset/3924.jpg
['Ocotea indecora']
/home/haroldle/Downloads/dataset/1408.jpg
['Cordia goeldiana']


In [24]:
# train lr stacked model

from sklearn.linear_model import LogisticRegression
import SonaliModel

sonali = SonaliModel.SonaliModel()
le = LeModel.LeModel()

df = pd.read_csv('flora.csv')
df['filename'] = '/home/haroldle/Downloads/dataset/' + df['ID'].astype(str).str.zfill(4) + '.jpg'

# taking class id minus one, following le's prediction output
lb = pd.read_csv("labels.csv")
df['y'] = df['ID'].astype(str).str.zfill(4).str[:2].astype(int)-1 

import random

df['random'] = [random.randrange(0, 46) for iter in range(df.shape[0])]

predictions = []
sonaliPred = []
for idx in range(df.shape[0]):
    #print(test['filename'].values[idx])
    im = cv2.imread(df['filename'].values[idx])
    predictions.append(lb[lb["SPECIES"]==le.predict(im)[0]]["ID"].values[0])
    
    if(sonali.predict(im)[0] == "Acrocarpus-fraxinifolius"):
        sonaliPred.append(lb[lb["SPECIES"]=="Acrocarpus fraxinifolius"]["ID"].values[0])
    else:
        sonaliPred.append(lb[lb["SPECIES"]==sonali.predict(im)[0]]["ID"].values[0])
    
df['le'] = predictions
df['sonali'] = predictions

test = df.loc[df['SET']=='test']
    
#clf = LogisticRegression(random_state=0).fit(X, y)

In [25]:
df.head(5)

,Unnamed: 0,SPECIES,ID,SET,filename,y,random,le,sonali
0,297,Manilkara elata,3135,train,/home/haroldle/Downloads/dataset/3135.jpg,30,21,30,30
1,137,Mezilaurus itauba,3307,train,/home/haroldle/Downloads/dataset/3307.jpg,32,18,32,32
2,1138,Araucaria angustifolia,223,train,/home/haroldle/Downloads/dataset/0223.jpg,1,2,1,1
3,1333,Swietenia macrophylla,4509,train,/home/haroldle/Downloads/dataset/4509.jpg,44,21,44,44
4,186,Machaerium paraguariense,2929,train,/home/haroldle/Downloads/dataset/2929.jpg,28,31,28,28


In [67]:
from sklearn.ensemble import RandomForestClassifier
import pickle
lr = LogisticRegression(random_state=0, solver = 'lbfgs', max_iter=10000).fit(test[['le','random','sonali']], test['y'])
rf = RandomForestClassifier(random_state=0).fit(test[['le','random','sonali']], test['y'])

f = open("/home/haroldle/pythonProject/brazilflora/EnsembleLogistic.pickle", "wb")
f.write(pickle.dumps(lr))

f = open("/home/haroldle/pythonProject/brazilflora/EnsembleRandomForest.pickle", "wb")
f.write(pickle.dumps(rf))


7098488

In [68]:
import pickle
model = pickle.load(open("EnsembleLogistic.pickle", "rb"))
df['lr'] = model.predict(df[['le','random','sonali']])

df['rf'] = rf.predict(df[['le','random','sonali']])


In [69]:
#sum(df.loc[df['SET']=='validation']['y'] == df.loc[df['SET']=='validation']['stacked'])
df.head(20)

,Unnamed: 0,SPECIES,ID,SET,filename,y,random,le,sonali,lr,rf
0,297,Manilkara elata,3135,train,/home/haroldle/Downloads/dataset/3135.jpg,30,21,30,30,33,31
1,137,Mezilaurus itauba,3307,train,/home/haroldle/Downloads/dataset/3307.jpg,32,18,32,32,33,19
2,1138,Araucaria angustifolia,223,train,/home/haroldle/Downloads/dataset/0223.jpg,1,2,1,1,1,3
3,1333,Swietenia macrophylla,4509,train,/home/haroldle/Downloads/dataset/4509.jpg,44,21,44,44,44,44
4,186,Machaerium paraguariense,2929,train,/home/haroldle/Downloads/dataset/2929.jpg,28,31,28,28,32,28
5,1407,Mimosa scabrella,3515,train,/home/haroldle/Downloads/dataset/3515.jpg,34,24,34,34,32,5
6,820,Inga vera,2718,train,/home/haroldle/Downloads/dataset/2718.jpg,26,20,26,26,27,25
7,623,Eucalyptus sp,1821,train,/home/haroldle/Downloads/dataset/1821.jpg,17,26,34,34,32,17
8,1501,Ocotea indecora,3924,train,/home/haroldle/Downloads/dataset/3924.jpg,38,15,38,38,37,37
9,1436,Cordia goeldiana,1408,train,/home/haroldle/Downloads/dataset/1408.jpg,13,7,13,13,5,13
